<a href="https://colab.research.google.com/github/MINJIN82/-Ubion/blob/main/type2_2nd(blank)_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 작업형2
- 빅데이터 분석기사 2회 실기
- 데이터 다운로드 : https://www.kaggle.com/prachi13/customer-analytics
- 데이터 파일 명 : Train.csv (파일명이 다를 경우 파일명을 수정해주세요)
- (데이터가 삭제될 시 수업노트에 유사 데이터 셋을 올려두겠습니다)


## 제품 배송 시간에 맞춰 배송되었는지 예측모델 만들기
- 제공된 데이터
    - X_train.csv
    - y_train.csv
    - X_test.csv

- 학습용 데이터 (X_train, y_train)을 이용하여 배송 예측 모형을 만든 후, 이를 평가용 데이터(X_test)에 적용하여 얻은 예측(시간에 맞춰 도착하지 않을 확률)값을 다음과 같은 형식의 CSV파일로 생성하시오(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점)
- 0 정시 도착, 1 정시 도착하지 않음

```
ID,Reached.on.Time_Y.N
3500,0.267
3501,0.578
3502,0.123

```

- 출력을 원하실 경우 print() 함수 활용
- 예시) print(df.head())
- getcwd(), chdir() 등 작업 폴더 설정 불필요
- 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

- 데이터 파일 읽기 예제

```
# 데이터 불러오기
import pandas as pd
X_test = pd.read_csv("data/X_test.csv")
X_train = pd.read_csv("data/X_train.csv")
y_train = pd.read_csv("data/y_train.csv")
```

- csv파일 생성 예시

```
pd.DataFrame({'ID': X_test['cust_id'], 'Reached.on.Time_Y.N': pred}).to_csv('003000000.csv', index=False)
```


In [ ]:
# 시험환경 세팅 3개 파일 생성 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name

    if null_name != "":
        df[df == null_name] = np.nan

    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2022)
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test

df = pd.read_csv("Train.csv") # 파일명이 다를 경우 파일명을 수정해주세요
X_train, X_test, y_train, y_test = exam_data_load(df, target='Reached.on.Time_Y.N', id_name='ID')
X_train.to_csv("X_train.csv", index=False)
y_train.to_csv("y_train.csv", index=False)
X_test.to_csv("X_test.csv", index=False)

In [ ]:
# 데이터 불러오기
import pandas as pd
X_test = pd.read_csv("X_test.csv")
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")

X_train.shape, y_train.shape, X_test.shape

((8799, 11), (8799, 2), (2200, 11))

## EDA

In [ ]:
# 데이터 확인(X_train)
X_train.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,8440,B,Ship,4,1,284,4,low,F,4,4795
1,10849,D,Ship,3,2,201,4,medium,M,10,5565
2,9478,B,Ship,4,4,181,3,medium,F,9,4425
3,2968,B,Ship,6,2,236,4,low,F,61,3140
4,6964,B,Road,5,1,282,4,low,F,9,4766


In [ ]:
# 데이터 확인(X_test)
X_test.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,8285,C,Ship,6,5,229,8,medium,F,2,4271
1,10192,B,Ship,7,3,306,4,low,F,4,1630
2,8675,C,Ship,4,5,148,3,medium,F,5,5216
3,5753,C,Flight,4,4,236,3,low,M,10,4347
4,448,B,Ship,3,4,194,3,medium,M,3,2169


In [ ]:
# 데이터 확인(y_train)
y_train.head()

,ID,Reached.on.Time_Y.N
0,8440,1
1,10849,0
2,9478,1
3,2968,1
4,6964,0


In [ ]:
# 결측치 확인 train
X_train.isnull().sum()
y_train.isnull().sum()

ID                     0
Reached.on.Time_Y.N    0
dtype: int64

In [ ]:
# 결측치 확인 test
X_test.isnull().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64

In [ ]:
# 타겟(레이블) 확인 0 정시 도착, 1 정시 도착하지 않음
y_train['Reached.on.Time_Y.N'].value_counts()

1    5236
0    3563
Name: Reached.on.Time_Y.N, dtype: int64

In [ ]:
# 데이터 타입 확인
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8799 entries, 0 to 8798
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   8799 non-null   int64 
 1   Warehouse_block      8799 non-null   object
 2   Mode_of_Shipment     8799 non-null   object
 3   Customer_care_calls  8799 non-null   int64 
 4   Customer_rating      8799 non-null   int64 
 5   Cost_of_the_Product  8799 non-null   int64 
 6   Prior_purchases      8799 non-null   int64 
 7   Product_importance   8799 non-null   object
 8   Gender               8799 non-null   object
 9   Discount_offered     8799 non-null   int64 
 10  Weight_in_gms        8799 non-null   int64 
dtypes: int64(7), object(4)
memory usage: 756.3+ KB


In [ ]:
# 기초 통계 획인
X_train.describe()

,ID,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms
count,8799.000000,8799.000000,8799.000000,8799.000000,8799.000000,8799.000000,8799.000000
mean,5514.696556,4.051369,2.991817,209.800318,3.561882,13.293215,3633.145130
std,3175.107399,1.141739,1.411938,48.186779,1.518472,16.112977,1637.717073
min,1.000000,2.000000,1.000000,96.000000,2.000000,1.000000,1001.000000
25%,2749.500000,3.000000,2.000000,169.000000,3.000000,4.000000,1837.000000
50%,5519.000000,4.000000,3.000000,213.000000,3.000000,7.000000,4152.000000
75%,8253.000000,5.000000,4.000000,250.000000,4.000000,10.000000,5055.000000
max,10999.000000,7.000000,5.000000,310.000000,10.000000,65.000000,7846.000000


In [ ]:
# 기초 통계 획인 object
X_train.describe(include = "object")

,Warehouse_block,Mode_of_Shipment,Product_importance,Gender
count,8799,8799,8799,8799
unique,5,3,3,2
top,F,Ship,low,F
freq,2929,5972,4225,4450


In [ ]:
# 기초 통계 획인 object
X_test.describe(include = "object")

,Warehouse_block,Mode_of_Shipment,Product_importance,Gender
count,2200,2200,2200,2200
unique,5,3,3,2
top,F,Ship,low,M
freq,737,1490,1072,1105


In [ ]:
# [Tip] object 타입 고유값 개수 확인
X_train[['Warehouse_block','Mode_of_Shipment','Product_importance','Gender']].nunique()

Warehouse_block       5
Mode_of_Shipment      3
Product_importance    3
Gender                2
dtype: int64

In [ ]:
# X데이터와 y데이터 합치기
df = pd.concat([X_train,y_train['Reached.on.Time_Y.N']],axis=1)
df.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,8440,B,Ship,4,1,284,4,low,F,4,4795,1
1,10849,D,Ship,3,2,201,4,medium,M,10,5565,0
2,9478,B,Ship,4,4,181,3,medium,F,9,4425,1
3,2968,B,Ship,6,2,236,4,low,F,61,3140,1
4,6964,B,Road,5,1,282,4,low,F,9,4766,0


In [ ]:
# idea? 할인율이 도착에 영향을 미칠까?
cond = df['Discount_offered'] > 10
sum(cond)

2104

## 데이터 전처리 및 피처엔지니어링

In [ ]:
# 라벨 인코딩
from sklearn.preprocessing import LabelEncoder
cols = X_train.select_dtypes(include = 'object').columns

for col in cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])

X_train.head()



,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,8440,1,2,4,1,284,4,1,0,4,4795
1,10849,3,2,3,2,201,4,2,1,10,5565
2,9478,1,2,4,4,181,3,2,0,9,4425
3,2968,1,2,6,2,236,4,1,0,61,3140
4,6964,1,1,5,1,282,4,1,0,9,4766


In [ ]:
# trainID 삭제, testID 값만 옮겨둠
X_train = X_train.drop(['ID'],axis=1)
X_testID = X_test.pop('ID')
X_test.head()

,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,2,2,6,5,229,8,2,0,2,4271
1,1,2,7,3,306,4,1,0,4,1630
2,2,2,4,5,148,3,2,0,5,5216
3,2,0,4,4,236,3,1,1,10,4347
4,1,2,3,4,194,3,2,1,3,2169


## 모델 및 평가

In [ ]:
import sklearn.tree
print(sklearn.tree.__all__)

['BaseDecisionTree', 'DecisionTreeClassifier', 'DecisionTreeRegressor', 'ExtraTreeClassifier', 'ExtraTreeRegressor', 'export_graphviz', 'plot_tree', 'export_text']


In [ ]:
# 라이브러리 불러오기
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import roc_auc_score

In [ ]:
# 검증 데이터 분리
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train,
                                            y_train['Reached.on.Time_Y.N'],
                                            test_size = 0.2,
                                            random_state = 2022)
X_tr.shape, X_val.shape, y_train.shape, y_test.shape

((7039, 10), (1760, 10), (8799, 2), (2200, 2))

In [ ]:
# 로지스틱 회귀
model = LogisticRegression(random_state=2022)
model.fit(X_tr,y_tr)
pred = model.predict_proba(X_val)
print(roc_auc_score(y_val,pred[:,1]))

0.7297546090137466


In [ ]:
# 의사결정나무
model = DecisionTreeClassifier(random_state = 2022)
model.fit(X_tr,y_tr)
pred = model.predict_proba(X_val)

print(roc_auc_score(y_val,pred[:,1]))

0.63291087659106


In [ ]:
# 랜덤포레스트
model = RandomForestClassifier(random_state = 2022)
model.fit(X_tr, y_tr)
pred = model.predict_proba(X_val)

print(roc_auc_score(y_val,pred[:,1]))


0.7554981502487046


In [ ]:
# XGBOOST
model= XGBClassifier(random_state = 2022)
model.fit(X_tr,y_tr)
pred = model.predict_proba(X_val)

print(roc_auc_score(y_val,pred[:,1]))

0.7501733515569575


## 예측 및 제출

In [ ]:
# 예측
pred = model.predict_proba(X_test)
pred

array([[0.6690933 , 0.3309067 ],
       [0.6749251 , 0.32507488],
       [0.658461  , 0.34153906],
       ...,
       [0.38199848, 0.6180015 ],
       [0.6544448 , 0.3455552 ],
       [0.6882974 , 0.31170258]], dtype=float32)

In [ ]:
# 데이터프레임 만들기
submit = pd.DataFrame({
                        "ID":X_testID,
                        "Reached.on.Time_Y.N" : pred[:,1]
})

In [ ]:
# 데이터 확인
submit.head()

,ID,Reached.on.Time_Y.N
0,8285,0.330907
1,10192,0.325075
2,8675,0.341539
3,5753,0.457580
4,448,0.992162


In [ ]:
# CSV파일 만들기
submit.to_csv("0000.csv", index = False)

##  제대로 제출 되었는지 확인

In [ ]:
pd.read_csv("0000.csv")

,ID,Reached.on.Time_Y.N
0,8285,0.330907
1,10192,0.325075
2,8675,0.341539
3,5753,0.457580
4,448,0.992162
...,...,...
2195,7193,0.472599
2196,4445,0.175447
2197,7390,0.618001
2198,9977,0.345555


## 체점
- 수험자는 알 수 없는 부분임

In [ ]:
you = pd.read_csv("0000.csv")
print("당신의 score는?:",roc_auc_score(y_test['Reached.on.Time_Y.N'], you['Reached.on.Time_Y.N']))


# 당신의 score는?: 0.7365117469492115

당신의 score는?: 0.7326907622202024


In [ ]:
# 당신의 score는?: 0.7396611568179091